**MetalPy** + SAP2000
-----------------

Este notebook apresenta um exemplo de aplicação do pacote **MetalPy**, integrando-o ao software de análise estrutural SAP2000.

Neste exemplo **MetalPy** é utilizado para fazer a verificação das barras de um modelo de treliça apresentado na figura 1. A treliça é formada por 33 barras, todas elas constítuidas de perfis tubo retangulares laminados e está submetido a um carregamento de aproximadamente 50kN em cada um de seus nós.

<h1 align="center">
<img src="img\\modelo_sap_carregamento.png">
</h1><br>

A treliça apresentada é formado por 4 diferentes tipos de perfis como mostrado na figura 2, todos eles de aço ASTM A992:

<h1 align="center">
<img src="img\\modelo_sap_secoes.png">
</h1><br>

 * Barras do banzo superior - perfil TQ160X160X8
 * Barras do banzo inferior - perfil TQ100X80X8.8
 * Barras dos motantes - perfil TR150X120X6.4
 * Barras das diagonais -  perfil TQ100X80X8.8

A obtenção dos esforços solicitantes é realizada através da análise estrutural realizado no SAP2000 e a verificação dos perfis é realizada com as funções do pacote **Metalpy** com os critérios de verificação da NBR8800:2008.

In [ ]:
#Importação do scrip de inicialização do SAP2000
from iniciar_sap import IniciarSAP

#Importando as classes necessárias do pacote Metalpy
from metalpy.perfis import TuboRet
from metalpy.material import Aco
from metalpy.normas import NBR8800

In [ ]:
# Iniciando o SAP2000
modelo_sap = IniciarSAP()
modelo_sap.InitializeNewModel()

#Abrindo o Modelo de Analise no Sap
FileName = 'C:\CSiAPIexample\TreliçaModelo.sdb' #Conferir o Diretório onde o arquivo do Modelo está Salvo
ret = modelo_sap.File.OpenFile(FileName)

In [ ]:
# Verificando o se o modelo está com a análise processada
if not modelo_sap.GetModelIsLocked():
    
    # Rodando a análise caso ela não esteja processada
    modelo_sap.Analyze.RunAnalysis() 

In [158]:
#Obtendo o nome das barras da treliça do SAP2000
barras_sap = modelo_sap.FrameObj.GetNameList()[1]

for barra in barras_sap:

    # -----------------------------
    # OBTENÇÃO DOS DADOS DO SAP2000
    # -----------------------------

    # Obtendo o nome do perfil da barra
    perfil = modelo_sap.FrameObj.GetSection(barra)[0]

    # Obtendo as propriedades do material da barra
    mat_perfil_sap = modelo_sap.PropFrame.GetMaterial(perfil)[0]

    E, v = modelo_sap.PropMaterial.GetMPIsotropic(mat_perfil_sap)[:2]
    fy, fu = modelo_sap.PropMaterial.GetOSteel_1(mat_perfil_sap)[:2]

    # Obtendo o comprimento a as solicitções das barras
    modelo_sap.Results.Setup.SetCaseSelectedForOutput(barra)
    resultados = modelo_sap.Results.FrameForce(barra, 0)

    klx = kly = klz = resultados[2][-1] # Comprimento da barra
    Nsd = max(resultados[8]) # Esforço axial na barra

    # -------------------------------------
    # REALIZANDO A VERIFICAÇÃO COM METALPY
    # -------------------------------------

    aco = Aco(E, v, fy, fu, 0.6) # Criando o aço correspondente 
    perfil_mp = TuboRet(perfil, aco, und='cm') # Criando o perfil correspondente

    if Nsd > 0:
        Nrd = NBR8800.Ntrd_brt(perfil_mp)
        
    elif Nsd <= 0:
        Nrd = NBR8800.Ncrd(perfil_mp, klx, kly, klz)


    if Nrd >= abs(Nsd):
        print('Barra ' + barra + ': Ok!', ' - Nsd/Nrd=', abs(Nsd / Nrd))

    else:
        print('Barra ' + barra + ': ** NÃO PASSOU NA VERIFICAÇÃO**',' - Nsd/Nrd=', abs(Nsd / Nrd))

Barra 1: Ok!  - Nsd/Nrd= 0.9230705616857159
Barra 2: Ok!  - Nsd/Nrd= 0.6928467921042042
Barra 3: Ok!  - Nsd/Nrd= 0.4619945859832385
Barra 4: Ok!  - Nsd/Nrd= 0.2311423798622571
Barra 5: Ok!  - Nsd/Nrd= 0.11135980513054947
Barra 6: Ok!  - Nsd/Nrd= 0.2311423798622571
Barra 7: Ok!  - Nsd/Nrd= 0.4619945859832385
Barra 8: Ok!  - Nsd/Nrd= 0.69284679210422
Barra 9: Ok!  - Nsd/Nrd= 0.9230705616857295
Barra 10: Ok!  - Nsd/Nrd= 2.6652870061667693e-14
Barra 11: Ok!  - Nsd/Nrd= 0.4090087974046535
Barra 12: Ok!  - Nsd/Nrd= 0.45084859747604183
Barra 13: Ok!  - Nsd/Nrd= 0.701157938407974
Barra 14: Ok!  - Nsd/Nrd= 0.7728833099589132
Barra 15: Ok!  - Nsd/Nrd= 0.8764474230099926
Barra 16: Ok!  - Nsd/Nrd= 0.9661041374486314
Barra 17: Ok!  - Nsd/Nrd= 0.9348772512106515
Barra 18: Ok!  - Nsd/Nrd= 0.9661041374486066
Barra 19: Ok!  - Nsd/Nrd= 0.9348772512106515
Barra 20: Ok!  - Nsd/Nrd= 0.7728833099588938
Barra 21: Ok!  - Nsd/Nrd= 0.8764474230099841
Barra 22: Ok!  - Nsd/Nrd= 0.4508485974760218
Barra 23: Ok!  -